<a href="https://colab.research.google.com/github/AndreaMazzoleniOOI/OOI_Python-courses/blob/main/Cat_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import sys

drive.mount('/src', force_remount=True)
sys.path.insert(0, './content/src')

Mounted at /src


In [2]:
import glob
import cv2 as cv
import random
import matplotlib.pyplot as plt
from src.create_dataset import Dataset
from src.model import ModelParams
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras import backend
backend.set_image_data_format('channels_last')
import os

[]
True


In [3]:
paths_dir = [f'/content/img/{p}' for p in os.listdir('/content/img')]

data = Dataset(dir_paths=paths_dir, size=50)
data.create_labels_from_paths()
data.info()

Dataset origin: ['/content/img/dog', '/content/img/cat', '/content/img/.ipynb_checkpoints']
Shape order: NWHC
Dataset: 24
Size of x_train: (16, 50, 50, 3)
Size of y_train: (16,)
Size of x_test: (8, 50, 50, 3)
Size of y_test: (8,)


In [8]:
params = ModelParams(file_path='/content/config/system_configuration.json')
print(data.x_train.shape[1:])
model = Sequential()
model.add(Convolution2D(params.nb_filters, params.nb_conv, padding='same', input_shape=data.x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(params.nb_filters, params.nb_conv))
model.add(MaxPooling2D(pool_size=(params.nb_pool, params.nb_pool)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(params.nb_dense))
model.add(Dropout(0.5))
model.add(Dense(params.nb_classes))
model.add(Activation('sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.summary()
# Train
epochs = 1
batch = 5

(50, 50, 3)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 50, 50, 2)         56        
                                                                 
 activation_2 (Activation)   (None, 50, 50, 2)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 48, 48, 2)         38        
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 24, 24, 2)         0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 24, 24, 2)         0         
                                                                 
 flatten_1 (Flatten)         (None, 1152)              0         
                                          

In [9]:
model.fit(data.x_train, data.y_train, batch_size=batch, epochs=epochs, validation_data=(data.x_test, data.y_test))

4/4 [==============================] - 2s 132ms/step - loss: 0.7837 - accuracy: 0.4375 - val_loss: 0.7015 - val_accuracy: 0.5000
